In [1]:
import pandas as pd
import io
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.feature_selection import RFE


In [2]:

# Load dataset from CSV file
data = pd.read_csv('processed_Infilteration_2.csv')
data= data.drop('Timestamp',axis=1)

# Separate the feature columns from the label column
X = data.drop('Label', axis=1)
y = data['Label']


In [3]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.6, random_state=42)


In [5]:
import time

svm = SVC(kernel='linear')
num_features = 30
step = 20
rfe = RFE(estimator=svm, n_features_to_select=num_features, step=step)

start_time = time.time()
print("RFE feature selection progress:")
for i in range(1, num_features+1, step):
    rfe.n_features_to_select = i
    rfe.fit(X_train, y_train)
    selected_features = X_train.columns[rfe.support_]
    accuracy = svm.fit(X_train[selected_features], y_train).score(X_test[selected_features], y_test)
    elapsed_time = time.time() - start_time
    remaining_time = (num_features-i)*elapsed_time/i if i > 0 else 0
    print("Number of features selected: %d, Accuracy: %.3f, Elapsed time: %.2f s, Remaining time: %.2f s" % (i, accuracy, elapsed_time, remaining_time))

selected_features = X_train.columns[rfe.support_]
accuracy = svm.fit(X_train[selected_features], y_train).score(X_test[selected_features], y_test)
print("Selected Features: %s, Accuracy: %.3f" % (selected_features.tolist(), accuracy))


RFE feature selection progress:


In [ ]:
# Train a final SVM model using the selected features
X_train_selected = X_train[selected_features]
X_test_selected = X_test[selected_features]
svm_final = SVC(kernel='linear')
svm_final.fit(X_train_selected, y_train)



In [ ]:
# Create a heatmap of the feature weights
coef = svm_final.coef_
coef_df = pd.DataFrame(coef, columns=selected_features)
sns.heatmap(coef_df, cmap='coolwarm')
plt.title('Feature Weights Heatmap')
plt.show()